In [75]:
# stores the results of all the steps up to maxsize anything over maxsize overwrites oldest data

# setting to 0 disables it

class Result_Cache:
    def __init__(self, max_size):
        self.max_size = max_size
        self.size = 0
        self.results = []
        self.to_replace = 0

    def append(self, result):
        
        if(self.max_size != 0):
            if(self.size < self.max_size):
                
                self.results.append(result)
                self.size = self.size + 1
               
            else:
                self.results[self.to_replace] = result
                self.to_replace = ( self.max_size - ( self.to_replace + 1 )) % self.max_size
            

In [81]:
def check_external(step):
    return step.__name__ == "is_external" and step.__module__ == "pipe"

In [84]:
cache = Result_Cache(2)
cache.append(1)
cache.append(2)
cache.append(3)
cache.append(4)
cache.append(5)
# cache.append(6)
# cache.append(7)
# cache.append(8)
# cache.append(9)
# cache.append(10)
cache.results

[5, 4]

In [ ]:
# if the step is external rerun it and compare its value to the value in the cache

# how do I know if its value is in the cache if we jump forward?

# maxsize cannot be smaller than the length of the pipe otherwise there is no benefit because there would be no data
# to compare against 

# how would we compare non hashable values such as dask pointers?

# this would also cause a memory explosion if all the steps manipulate the same multi gigabyte data due to its
# compounding nature

# solution: only store the last result, and any result that comes just before a external step

In [48]:
step_queue = []

# serves as a marker for caching system
# during run check to see for any steps with the __name__ is_external from module pipe mark index
def external(step):
    def is_external(data):
        return step(data)
    return is_external

In [45]:
import inspect
from sys import version_info

def resolve_get_args():
    if(version_info[0] == 2):
        return inspect.getargspec
    else:
        return inspect.getfullargspec


class Pipe:
    
    def __init__(self, *steps, **kwargs):
        self.name = kwargs.get("name", None)
        
        steps = list(steps)
        
#         steps = fix_pipe_arg(*steps)

        if callable(steps[0]):
            self.steps = steps
            self.start_value = None
        else:
            self.start_value = steps[0]
            self.steps = steps[1:]
            
    def append(self, *steps):
        self.steps += steps
        
    def insert(self, index, step):
        self.steps = self.steps.insert(index, step)
        
    def replace(self, index, step):
        self.steps[index] = step

    def open(self, data=None):
        
        def get_result(step, data):
            args = get_args(step).args
            num_args = len(args)
            
            if num_args == 1 or (num_args == 2 and args[0] == "self"):
                return step(data)
                
            elif num_args > 1:
                return step(*data)
            
            return step()
        
        prev_result = None
        get_args = resolve_get_args()

        if self.start_value != None:
            if(data == None):
                data = self.start_value
            else:
                print("WARNING! you put a raw value at the top of your pipe and you put " + str(data) + " in the opening of the pipe the value at the start of the pipe has been overwritten by the passed in value. You may want to get rid of that value at the top of the pipe to get rid of this message\n")
            
    
        if data != None:
            
            prev_result = get_result(self.steps[0], data)
        
        else:
            prev_result = self.steps[0]()
     
        for step in self.steps[1:]:
            prev_result = get_result(step, prev_result)

        # the use is multithreading so add the result to the pool of data
        if(self.name != None):
            global pool
            pool.update({self.name: prev_result})

        return prev_result
        

In [46]:
def blah():
    return "hi"

testee = Pipe(blah)

test = Pipe(testee.open)

In [47]:
test.open("hi")